BOT TOKEN (BOTPLUG): MTI1NzM5MzgzNTAzNDY3MzM1Ng.GPYvu6.yglKQE6mvDXzP_Qw3GuF5A7ClJL8vP9haifjaE
CHANNEL ID: 1257050316059312210

!add_stock <ticker>: Adds a stock to the list of tracked stocks.
!remove_stock <ticker>: Removes a stock from the list of tracked stocks.
!list_stocks: Lists all tracked stocks.
!show_stock <ticker>: Shows the current price and a graph of the stock's price over the last month.

In [60]:
import discord
from discord.ext import commands, tasks
import yfinance as yf
import nest_asyncio
import asyncio
import matplotlib.pyplot as plt
import io

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Function to get stock price
def get_stock_price(ticker):
    stock = yf.Ticker(ticker)
    price = stock.history(period='1d')['Close'].iloc[-1]
    return price

# Function to get historical stock prices
def get_historical_prices(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period='1mo')  # Get one month of data
    return hist

# Function to plot the stock prices
def plot_stock_prices(ticker, hist):
    plt.figure(figsize=(10, 5))
    plt.plot(hist.index, hist['Close'], label='Close Price')
    plt.title(f'{ticker} Stock Price Over Last Month')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend()
    plt.grid(True)
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close()
    return buf

# Set up the Discord bot
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

# Global variable to store tickers
tracked_stocks = []

# Discord bot command to add a stock to track
@bot.command(name="add_stock")
async def add_stock(ctx, ticker):
    global tracked_stocks
    if ticker.upper() not in tracked_stocks:
        tracked_stocks.append(ticker.upper())
        await ctx.send(f"Added {ticker.upper()} to tracked stocks.")
    else:
        await ctx.send(f"{ticker.upper()} is already being tracked.")

# Discord bot command to remove a stock from tracking
@bot.command(name="remove_stock")
async def remove_stock(ctx, ticker):
    global tracked_stocks
    if ticker.upper() in tracked_stocks:
        tracked_stocks.remove(ticker.upper())
        await ctx.send(f"Removed {ticker.upper()} from tracked stocks.")
    else:
        await ctx.send(f"{ticker.upper()} is not being tracked.")

# Discord bot command to list tracked stocks
@bot.command(name="list_stocks")
async def list_stocks(ctx):
    global tracked_stocks
    if tracked_stocks:
        await ctx.send("Tracked stocks:\n" + "\n".join(tracked_stocks))
    else:
        await ctx.send("No stocks are being tracked currently.")

# Discord bot command to show stock prices and graph
@bot.command(name="show_stock")
async def show_stock(ctx, ticker):
    ticker = ticker.upper()
    if ticker not in tracked_stocks:
        await ctx.send(f"{ticker} is not being tracked. Use !add_stock {ticker} to track it.")
        return

    try:
        # Get current price
        price = get_stock_price(ticker)
        await ctx.send(f"The current price of {ticker} is ${price:.2f}")

        # Get historical prices
        hist = get_historical_prices(ticker)
        graph = plot_stock_prices(ticker, hist)
        
        # Send the graph
        await ctx.send(file=discord.File(graph, f"{ticker}_price.png"))
    except Exception as e:
        await ctx.send(f"Failed to fetch data for {ticker}: {e}")

# Task to fetch and send stock prices periodically
@tasks.loop(minutes=1)
async def fetch_and_send_stock_prices():
    global tracked_stocks
    channel = bot.get_channel(1257050316059312210)  # Replace with your Discord channel ID
    if tracked_stocks:
        for ticker in tracked_stocks:
            try:
                price = get_stock_price(ticker)
                await channel.send(f"The current price of {ticker} is ${price:.2f}")
            except Exception as e:
                await channel.send(f"Failed to fetch price for {ticker}: {e}")

# Start the task when the bot is ready
@bot.event
async def on_ready():
    print(f"Logged in as {bot.user} (ID: {bot.user.id})")
    print("Bot is ready to receive commands!")
    fetch_and_send_stock_prices.start()
    print("Started stock price fetcher task.")
    print("------")

# Run the bot
async def main():
    print("Starting the bot...")
    async with bot:
        await bot.start('MTI1NzM5MzgzNTAzNDY3MzM1Ng.GPYvu6.yglKQE6mvDXzP_Qw3GuF5A7ClJL8vP9haifjaE')  # Replace with your Discord bot token

# Check if running in a Jupyter notebook
try:
    get_ipython()
    loop = asyncio.get_event_loop()
    loop.create_task(main())
except NameError:
    asyncio.run(main())


Starting the bot...
Logged in as BOT PLUG#6351 (ID: 1257393835034673356)
Bot is ready to receive commands!
Started stock price fetcher task.
------


APPL: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
